v2:
* Try to get rid of settingiwhoutcopywaring...nogo
* added platform check
* this checkpoint ran with a score of .75xx on leaderboard (.643 rf score)

v3:
* added the differential area (d_area).  Didn't help with the local scores (.001 lower) and lb was ~.0004 lower
* altered/removed some of the other catboost params and did some tuning.  .943 @ 4000, .77.  not much luck


v4:
### Adding max u_in feature helped score from  ~.75 to ~.65.
* removed catboost tuning parameter trials
* add mutual info function
    extremely slow - do later with reduced dataset
* max_time feature was helpful, so make max_u_in too, and check scores

* wish I could get away from rf

Pipeline is:
* reduce memory
* split into two sets
* add features: max_time, max_u_in, dt, u_in_slope, u_in_area



# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import platform
pd.options.mode.chained_assignment = None

# Functions

In [2]:
# function to get single breath (id must exist...there are some that are missing)
# also once removed u_out=1, lengths are not always 30 (some 28, etc)
def get_breath(df,my_id):
    return df[df.breath_id == my_id] 

In [3]:
def plot_breath(df,my_id):
    id1 = get_breath(df,my_id)
    r = id1.R.iloc[0]
    c = id1.C.iloc[0]
    plt.figure(figsize=(8,5))
    plt.plot(id1.pressure,label='pressure')
    plt.plot(id1.u_in,label='u_in')
    plt.title(f'Pressure and u_in for Breath id={my_id}, R={r}, C={c}')
    plt.legend();

# Load Files

In [4]:
%%time
# Load files
if platform.system() == 'Windows' and platform.release() == '7':
    drive = 'F'
elif platform.system() == 'Windows' and platform.release() == '10':
    drive = 'C'
    
train = pd.read_csv(drive + r':\Sync\Work\Kaggle Competitions\Ventilator Pressure Prediction\Data\train.csv')
test = pd.read_csv(drive + r':\Sync\Work\Kaggle Competitions\Ventilator Pressure Prediction\Data\test.csv')

y = train['pressure']


Wall time: 4.91 s


# Memory Reduction

In [5]:
# down convert columns to save memory...
def reduce_memory(df):
    
    print(f'Initial memory usage: {df.memory_usage().sum()/1024**2:0.1f}GB')
    df['id'] = df['id'].astype(np.int32)
    df['breath_id'] = df['breath_id'].astype(np.int32)
    df['R'] = df['R'].astype(np.int8)  #or OHC?
    df['C'] = df['C'].astype(np.int8)
    df['u_out'] = df['u_out'].astype(np.int8)
    df['u_in'] = df['u_in'].astype(np.float32)
    df['time_step'] = df['time_step'].astype(np.float32)
    print(f'New memory usage: {df.memory_usage().sum()/1024**2:0.1f}GB')
    
    # somewhere else I printed out a reduction as well.
    
#     for col in test.columns:
#         print(test[col].dtype)

In [6]:
print('Train:')
reduce_memory(train)
print('Test:')
reduce_memory(test)


Train:
Initial memory usage: 368.4GB
New memory usage: 155.4GB
Test:
Initial memory usage: 214.9GB
New memory usage: 72.9GB


# Split data into inhalitory and exhalitory phase (only scored on inhale)

In [7]:
train_in = train[train.u_out == 0]
test_in = test[test.u_out == 0]
y_in = train_in['pressure']

train_out = train[train.u_out == 1]
test_out = test[test.u_out == 1]

#  Add Features

    1. Apply lag shift (tested shift =1 ) 
        Shift = 2 performs better (2.37 vs. 2.0x)
    2. Add differentials for dt and du_in
    3. Add integral column for d_uin



In [8]:
# first add max_u_in value
train_in["max_u_in"] = train_in.groupby("breath_id")["u_in"].transform("max")
test_in["max_u_in"] = test_in.groupby("breath_id")["u_in"].transform("max")


### Apply lag shift

In [9]:
shift = 1
# apply lag shift in training set
u_in_lag = train_in.u_in.shift(shift,fill_value=0)
train_in.loc[:,'u_in_lag'] = u_in_lag.copy()
train_in.drop(['u_in'],axis=1,inplace=True)

# and for test set
u_in_lag = test_in.u_in.shift(shift,fill_value=0)
test_in.loc[:,'u_in_lag'] = u_in_lag.copy()
test_in.drop(['u_in'],axis=1,inplace=True)

In [10]:
# transition 1
#train_in[0:33]

In [11]:
# u_in leaking into next breath...number leaked = 
transitions = np.where(np.diff(train_in.breath_id) == 1)
transitions[0]
# for r in transitions[0]:
#     print(train_in.u_in_lag.iloc[r+1])

array([     29,      58,      90, ..., 2290785, 2290817, 2290876],
      dtype=int64)

In [12]:
# make an index of breaths
train_breath_idx = train_in.breath_id.unique()
test_breath_idx = test_in.breath_id.unique()
test_breath_idx

array([     0,      8,     11, ..., 125746, 125747, 125748])

### Add Differential, Integral, and other Features

In [13]:
# faster area calculation... need to compeletely verify behaviour
# time delta is identical to my dt
# area is idential to u_in_integ, but one step behind (can use to check things.)

train_in['dt'] = train_in['time_step'].diff()
train_in['dt'].fillna(0, inplace=True)  # no na values so not needed
train_in['dt'].mask(train_in['dt'] < 0,0,inplace=True) #train_in.groupby('breath_id')['dt'].mean(), inplace=True)  #makes 

train_in['d_area'] = train_in['dt'] * train_in['u_in_lag']
train_in['du_in_integ'] = train_in.groupby('breath_id')['d_area'].cumsum()#.shift(1,fill_value=0)
#train_in['max_time'] = train_in.groupby('breath_id')['time_step'].max()  # why doesn't this work?
    
train_in['du_in'] = train_in['u_in_lag'].diff()  # then look where time delta is 0 and make this 0 also
train_in['du_in'].mask(train_in['dt'] == 0, 0, inplace=True)
train_in['dt'].fillna(0, inplace=True)

# train_in.drop(['dt'],axis=1,inplace=True)
train_in.drop(['d_area'],axis=1,inplace=True)

# same feature additions with test
test_in['dt'] = test_in['time_step'].diff()
test_in['dt'].fillna(0, inplace=True)  # no na values so not needed
test_in['dt'].mask(test_in['dt'] < 0, 0, inplace=True)

test_in['d_area'] = test_in['dt'] * test_in['u_in_lag']
test_in['du_in_integ'] = test_in.groupby('breath_id')['d_area'].cumsum()#.shift(1,fill_value=0)
    
test_in['du_in'] = test_in['u_in_lag'].diff()  # then look where time delta is 0 and make this 0 also
test_in['du_in'].mask(test_in['dt'] == 0, 0, inplace=True)

# test_in.drop(['dt'],axis=1,inplace=True)
test_in.drop(['d_area'],axis=1,inplace=True)

### Mutual Info

In [14]:
# mutual info
discrete_features = train_in.dtypes == np.int8
discrete_features
#train_in.id.dtype

id             False
breath_id      False
R               True
C               True
time_step      False
u_out           True
pressure       False
max_u_in       False
u_in_lag       False
dt             False
du_in_integ    False
du_in          False
dtype: bool

In [15]:
# so use regression (for continuous features) but then specify which features are discrete
#too slow!!!!!  Maybe try later on reduced set
# from sklearn.feature_selection import mutual_info_regression

# def make_mi_scores(X, y, discrete_features):
#     mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
#     mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
#     return mi_scores

# mi_scores = make_mi_scores(train_in, y_in, discrete_features)
# mi_scores # show a few features with their MI scores

In [16]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

# plt.figure(dpi=100, figsize=(8, 5))
# plot_mi_scores(mi_scores)

In [17]:
# try this for getting max time
train_in["max_time"] = train_in.groupby("breath_id")["time_step"].transform("max")
test_in["max_time"] = test_in.groupby("breath_id")["time_step"].transform("max")



In [18]:
train_in.head()

,id,breath_id,R,C,time_step,u_out,pressure,max_u_in,u_in_lag,dt,du_in_integ,du_in,max_time
0,1,1,20,50,0.000000,0,5.837492,28.313036,0.000000,0.000000,0.000000,0.000000,0.987487
1,2,1,20,50,0.033652,0,5.907794,28.313036,0.083334,0.033652,0.002804,0.083334,0.987487
2,3,1,20,50,0.067514,0,7.876254,28.313036,18.383041,0.033862,0.625293,18.299707,0.987487
3,4,1,20,50,0.101542,0,11.742872,28.313036,22.509277,0.034028,1.391235,4.126236,0.987487
4,5,1,20,50,0.135756,0,12.234987,28.313036,22.808823,0.034213,2.171600,0.299545,0.987487


In [19]:
# %%time
# # slow way to do this but works for now
# max_time_by_id = train_in.groupby('breath_id')['time_step'].max()
# train_in['max_time'] = np.zeros(len(train_in))
# for i in train_breaths:
#     train_in.max_time[train_in.breath_id == i] = max_time_by_id[i]

In [20]:
# %%time
# # now for test
# max_time_by_id_test = test_in.groupby('breath_id')['time_step'].max()
# test_in['max_time'] = np.zeros(len(test_in))
# for i in test_breaths:
#     test_in.max_time[test_in.breath_id == i] = max_time_by_id_test[i]

In [21]:

train_in.head(35)

,id,breath_id,R,C,time_step,u_out,pressure,max_u_in,u_in_lag,dt,du_in_integ,du_in,max_time
0,1,1,20,50,0.000000,0,5.837492,28.313036,0.000000,0.000000,0.000000,0.000000,0.987487
1,2,1,20,50,0.033652,0,5.907794,28.313036,0.083334,0.033652,0.002804,0.083334,0.987487
2,3,1,20,50,0.067514,0,7.876254,28.313036,18.383041,0.033862,0.625293,18.299707,0.987487
3,4,1,20,50,0.101542,0,11.742872,28.313036,22.509277,0.034028,1.391235,4.126236,0.987487
4,5,1,20,50,0.135756,0,12.234987,28.313036,22.808823,0.034213,2.171600,0.299545,0.987487
5,6,1,20,50,0.169698,0,12.867706,28.313036,25.355850,0.033942,3.032233,2.547028,0.987487
6,7,1,20,50,0.203708,0,14.695562,28.313036,27.259867,0.034010,3.959346,1.904016,0.987487
7,8,1,20,50,0.237723,0,15.890699,28.313036,27.127485,0.034015,4.882080,-0.132381,0.987487
8,9,1,20,50,0.271776,0,15.539188,28.313036,26.807732,0.034054,5.794985,-0.319754,0.987487
9,10,1,20,50,0.305732,0,15.750094,28.313036,27.864716,0.033955,6.741141,1.056984,0.987487


In [ ]:
# saved prepared train_in and test_in


# Model

In [22]:
from sklearn.metrics import mean_absolute_error  #confusion_matrix, classification_report

In [23]:
# Split data - after all analysis is done
from sklearn.model_selection import train_test_split

train_in.drop(columns = ['pressure','id'], inplace = True)
#test = test.drop(columns = 'id', inplace = True)

X_train, X_valid, y_train, y_valid = train_test_split(train_in, y_in, train_size=0.8, test_size=0.2,
                                                      random_state=12)
X_test_in = test_in.drop(columns=['id'],inplace=False)

In [24]:
train_in

,breath_id,R,C,time_step,u_out,max_u_in,u_in_lag,dt,du_in_integ,du_in,max_time
0,1,20,50,0.000000,0,28.313036,0.000000,0.000000,0.000000,0.000000,0.987487
1,1,20,50,0.033652,0,28.313036,0.083334,0.033652,0.002804,0.083334,0.987487
2,1,20,50,0.067514,0,28.313036,18.383041,0.033862,0.625293,18.299707,0.987487
3,1,20,50,0.101542,0,28.313036,22.509277,0.034028,1.391235,4.126236,0.987487
4,1,20,50,0.135756,0,28.313036,22.808823,0.034213,2.171600,0.299545,0.987487
...,...,...,...,...,...,...,...,...,...,...,...
6035945,125749,50,10,0.834147,0,25.504196,2.650333,0.033421,7.905259,0.212046,0.967743
6035946,125749,50,10,0.867574,0,25.504196,1.869367,0.033427,7.967745,-0.780965,0.967743
6035947,125749,50,10,0.900917,0,25.504196,2.154414,0.033343,8.039580,0.285047,0.967743
6035948,125749,50,10,0.934309,0,25.504196,1.304434,0.033391,8.083138,-0.849980,0.967743


In [26]:
%%time
from catboost import CatBoostRegressor
# loop for manual type cv
#preds = []
for i in np.arange(1,2):
#     X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.8, test_size=0.2,
#                                                       random_state=i)
    model_cat = CatBoostRegressor(loss_function="MAE",
                               eval_metric="MAE",
                               task_type="GPU",
                               learning_rate=.6,
                               iterations=4000,
                               l2_leaf_reg=50,
                               random_seed=12,
                               #od_type="Iter",
                               depth=5,
                               #early_stopping_rounds=6500,
                               #border_count=64,
                               verbose=False
                              )
    model_cat.fit(X_train,y_train)
    pred_cat = model_cat.predict(X_valid)
    score_cat = mean_absolute_error(y_valid,pred_cat)
    #print(f'iters={i}, lr={j}, CatBoost MAE Score: {score_cat}')
    print(f'CatBoost MAE Score: {score_cat}')
    #preds.append(model_cat.predict_proba(X_test)[:,1])
    # 400, .6 = 3.976 
    # with deriv, integral
    # 400, .6 = 1.42
    # 4000,.6 = 1.21 (1.18 lb)
    # 8000,.6 = 1.12 (1.14 lb) maybe starting to overfit
    # shift = 1 (still need to review lining up)
    # 400, .6 = 1.22
    # 4000: 1.1277 
    # try shift = 2, = 1.23 with 4000,.6 (worse)
    
    # again...
    # shift 1 = 1.287 with 400, .6 (1.33 with older method..shifting integral better?)
    # shift 1 = 1.0773 with 4000, .6, so it is a little better than the 1.1227 with the older method
    # removed NaN from dt, integ and is a little worse. 1.079 ()
    # 1.1943 with max_time added and 400 iters, rt = .65!
    # adding d_area column, 400,.6 mae = 1.1987 (slightly higher)
    # 4000 = .9679  what was it previosly?  Forgot to run (or record).
    # Next, study of l2_leaf_reg 5-45 in 5 inc: virtually no change: 1.1947-1.195x
    # Removed some other parameters. 400, .77 = 1.653, 4000, .77 = .943
    # 4000, .057: .9404
    # try some more lr values at 4000 iters .2 to .8 in .01 increments.  Not much improvement compared to rf: .985-.935
    # Best prev: 16000, .71,45,20 = .86958   ...still not competitive with random forest


    # 400, .6,50,5 = 1.09! , 4000 = .833!  this is lower..._adding max_u_in helped!

CatBoost MAE Score: 0.8336802650062256
Wall time: 1min 12s


In [27]:
%%time
#random forest
from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_valid)
rf_mae = mean_absolute_error(pred_rf,y_valid)
print(f'Random Forest MAE Score: {rf_mae}')

# (n_estimators=100, max_depth=7,min_samples_leaf=0.06, random_state=12), mae=3.12775, lb score = 6.431(?)
# Why is random forest worse?  
# 10/17/21: Still worse after using only inhales
# defaults: runs out of memory!!!???
# (n_estimators=100, max_depth=7,min_samples_leaf=0.06, random_state=12) = 5.867
# reduced dtype sizes on ints: 
# 3.627 default, lb = 3.710
# changed criterion to mae...doesn't work...removed

# with dt and du_in, shift=1, 2.364 vs. lb of 2.37.  Beats catboost (400,.6)
# with dt and du_in, shift=2, Random Forest MAE Score: 2.058162936194095
# with dt and du_in, shift=3, Random Forest MAE Score: 2.03014981349922
# interesting shift 3 is a little better.

# deriv and integral, shift=1, .7934!!!
# but leaderboard was .8897
# try shift = 2, = .934 (a little lower)
# added max_time column, down to .6429!  
# added d_area column - actually increased a little to .64394, but possible that lb is better...maybe try it.
# added max_u_in (in addition to max_time), .55379

Random Forest MAE Score: 0.5537901151111227
Wall time: 29min 19s


# Final Model Submission

In [28]:
pred_final = model_rf.predict(X_test_in)
# add indexs to recombine with out preds
pred_final_s = pd.Series(pred_final,index=list(test_in.id))
pred_final_s.head()

1    6.460369
2    5.991453
3    7.208384
4    7.795406
5    9.239413
dtype: float64

In [29]:
# create outpreds = just ones
out_preds = np.ones(len(test_out))
i = list(test_out.id)
out_preds_s = pd.Series(out_preds,index = i)
out_preds_s

32         1.0
33         1.0
34         1.0
35         1.0
36         1.0
          ... 
4023996    1.0
4023997    1.0
4023998    1.0
4023999    1.0
4024000    1.0
Length: 2496435, dtype: float64

In [30]:
both = pred_final_s.append(out_preds_s).sort_index()
both.values

array([6.46036871, 5.9914534 , 7.20838354, ..., 1.        , 1.        ,
       1.        ])

In [31]:
output = pd.DataFrame({'id': test.id, 'pressure': both.values})
output.to_csv('submission.csv', index=False)
print("Submission saved!")

Submission saved!


## Post Analyze prediction vs. actuals and look for trends

In [ ]:
# %%time
# ### predictions vs. validation data
pred_rf_all = model_rf.predict(train_in)

In [ ]:
y_in.values

In [ ]:
#Create a dataframe with breath_id's to make plotting easier
post_analysis = pd.DataFrame({'breath_id': train_in.breath_id, 'prediction': pred_rf_all, 'acutal': y_in.values})
post_analysis['residual'] = pred_rf_all - y_in.values
post_analysis['R'] = train_in.R
post_analysis['C'] = train_in.C
post_analysis['u_in_lag'] = train_in.u_in_lag


In [ ]:
# post_analysis.head()

In [ ]:
# make this a function
breath_idx = train_in.breath_id.unique()
fig, ax = plt.subplots(6, 4, figsize=(25,30))
ax = ax.flatten()
for i in range(24):
    b_id = get_breath(post_analysis,breath_idx[i])
    r = b_id.R.iloc[0]
    c = b_id.C.iloc[0]
    ax[i].plot(b_id.prediction,label='prediction')
    ax[i].plot(b_id.acutal,label='actual')
    ax[i].set_title(f'RR Prediction and Actual: Shift=1, Breath id={i}, R={r}, C={c}')
    ax[i].legend();
    
plt.savefig('RndFrst_pred_actual_shift=1_tot_time.png')
